In [6]:
import os
import sys

from utilities.evaluate import f1_score
from utilities.file_utils import Utils as utils
from data.bioasq_dataset import BioASQDataset

from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords
#print(stopwords.words('english'))

words = stopwords.words('english')

## Load BioASQ dataset

In [4]:
# to learn tfidf weights
train_dataset = BioASQDataset('train', notebook = True)
testdev_dataset = BioASQDataset('test-dev', notebook = True)
test_dataset = BioASQDataset('test', notebook = True)

Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-train-factoid-5b.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-train-factoid-4b.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-train-factoid-6b.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-4b-2.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-5.json


...


Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-2.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-4b-5.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-1.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-5b-1.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-4b-1.json


## Learn TFIDF weights from context documents (train split) and Transform features

In [7]:
corpus = [d[-1] for d in tqdm(train_dataset)]
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

/Users/ordonez2/anaconda3/envs/domain_adaptation_QA/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


TfidfVectorizer()

In [19]:
test_dev_vectors = {}
i = 0
pbar = tqdm(total = len(testdev_dataset))

for d in testdev_dataset:
    pbar.update(1)
    ques, ans, ans_span, context = d
    test_dev_vectors[i] = {'ques_text' : ques, 
                            'gt_ans_text': ans, 
                            'gt_ans_span': ans_span,
                            'context_text': context}
    
    context_sent = nltk.sent_tokenize(context)
    #remove stopwords
    context_vec = {}
    context_vec1 = vectorizer.transform([' '.join([w.lower() for w in c if w not in set(words)]) for c in context_sent])
    
    for j, c in enumerate(context_sent):
        norm_ctx = ' '.join([w.lower() for w in c if w not in set(words)])
        if len(norm_ctx) < 1:
            continue
        context_vec[j] = context_vec1[j]
        
    
    test_dev_vectors[i]['context_vec'] = context_vec 
    ques_vec = vectorizer.transform([' '.join([q for q in ques.split() if q not in set(words)])])
    
    test_dev_vectors[i]['question_vec'] = ques_vec
    max_idx, max_score = -1, 0
    all_scores = {}
    
    for k, v in context_vec.items():
        sim_score = cosine_similarity(ques_vec, v)
        if sim_score > max_score:
            max_score = sim_score
            max_idx = k
            
        all_scores[k] = sim_score
        
    test_dev_vectors[i]['scores'] = all_scores
    test_dev_vectors[i]['pred_ans'] = context_sent[max_idx]
    i +=1
   

pbar.close()

/Users/ordonez2/anaconda3/envs/domain_adaptation_QA/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
test_dev_vectors = {}
i = 0
pbar = tqdm(total = len(test_dataset))

for d in test_dataset:
    pbar.update(1)
    ques, ans, ans_span, context = d
    test_dev_vectors[i] = {'ques_text' : ques, 
                            'gt_ans_text': ans, 
                            'gt_ans_span': ans_span,
                            'context_text': context}
    
    context_sent = nltk.sent_tokenize(context)
    #remove stopwords
    context_vec = {}
    context_vec1 = vectorizer.transform([' '.join([w.lower() for w in c if w not in set(words)]) for c in context_sent])
    
    for j, c in enumerate(context_sent):
        norm_ctx = ' '.join([w.lower() for w in c if w not in set(words)])
        if len(norm_ctx) < 1:
            continue
        context_vec[j] = context_vec1[j]
        
    
    test_dev_vectors[i]['context_vec'] = context_vec 
    ques_vec = vectorizer.transform([' '.join([q for q in ques.split() if q not in set(words)])])
    
    test_dev_vectors[i]['question_vec'] = ques_vec
    max_idx, max_score = -1, 0
    all_scores = {}
    
    for k, v in context_vec.items():
        sim_score = cosine_similarity(ques_vec, v)
        if sim_score > max_score:
            max_score = sim_score
            max_idx = k
            
        all_scores[k] = sim_score
        
    test_dev_vectors[i]['scores'] = all_scores
    test_dev_vectors[i]['pred_ans'] = context_sent[max_idx]
    i +=1
   

pbar.close()

/Users/ordonez2/anaconda3/envs/domain_adaptation_QA/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
def norm(text):
    return ' '.join([t.lower() for t in text.split() if t not in set(words)])

all_gt = [norm(v['gt_ans_text']) for k, v in test_dev_vectors.items()]
all_pred = [norm(v['pred_ans']) for k, v in test_dev_vectors.items()]


In [29]:
all_gt[0:10]

['sonidegib hedghog signalling pathway inhibitor.',
 'sonidegib hedghog signalling pathway inhibitor.',
 'denosumab (dmab) fully human monoclonal antibody receptor activator nuclear factor-κb ligand (rankl).',
 'denosumab (dmab) fully human monoclonal antibody receptor activator nuclear factor-κb ligand (rankl).',
 'denosumab (dmab) fully human monoclonal antibody receptor activator nuclear factor-κb ligand (rankl).',
 'denosumab (dmab) fully human monoclonal antibody receptor activator nuclear factor-κb ligand (rankl).',
 "eliglustat developed treatment gaucher's disease type 1.",
 "eliglustat developed treatment gaucher's disease type 1.",
 "eliglustat developed treatment gaucher's disease type 1.",
 "eliglustat developed treatment gaucher's disease type 1."]

In [30]:
all_pred[0:10]

['funding: novartis pharmaceuticals corporation.',
 'conclusions: this five-gene hh signature robustly identify hh-activated medulloblastoma may used preselect patients might benefit sonidegib treatment.',
 'in 2013, us food drug administration approved denosumab use patients recurrent/unresectable/metastatic giant-cell tumor bone patients surgery would morbid.',
 'according data, dmab may represent reasonable alternative bisphosphonates, particularly osteoporotic women suboptimal even poor adherence oral treatments expected.',
 'this may contribute substantially significant reduction nonvertebral fracture risk found long-term denosumab arm freedom extension trial years 4-7.',
 'rapid desensitization promising method delivery denosumab hypersensitivity reaction, considered osteoporosis treatment acceptable therapeutic alternatives available.',
 'conclusions: eliglustat may therapeutic option treating skeletal manifestations gd1.',
 'funding: genzyme, sanofi company.',
 'trial registrat

In [25]:
print('test_dev f1 score: {}'.format(f1_score(all_pred, all_gt)))

test_dev f1 score: 0.05045675252956613


In [31]:
print('test f1 score: {}'.format(f1_score(all_pred, all_gt)))

test f1 score: 0.0895694148136883
